In [ ]:
import random
import math
from compas.geometry import Point, Polyline
from compas.colors import Color
from compas_notebook.viewer import Viewer

## 1. Generate Random Points

Let's generate a cloud of random 2D points.

In [ ]:
def generate_points(n=50, range_x=(0, 100), range_y=(0, 100)):
    points = []
    for _ in range(n):
        x = random.uniform(*range_x)
        y = random.uniform(*range_y)
        points.append(Point(x, y, 0))
    return points

# Generate 30 points
points = generate_points(30)
print(f"Generated {len(points)} points.")

## 2. Graham Scan Algorithm

### Step 2.1: Find the Start Point
Find the point with the lowest Y coordinate. If there's a tie, choose the one with the lowest X coordinate. This point is guaranteed to be on the hull.

In [ ]:
def get_start_point(points):
    # Sort by Y, then by X
    return min(points, key=lambda p: (p.y, p.x))

start_point = get_start_point(points)
print(f"Start Point: {start_point}")

### Step 2.2: Sort Points by Polar Angle
Sort the remaining points based on the angle they make with the start point and the X-axis.

In [ ]:
def polar_angle(p0, p1):
    return math.atan2(p1.y - p0.y, p1.x - p0.x)

def distance_sq(p0, p1):
    return (p1.x - p0.x)**2 + (p1.y - p0.y)**2

# Sort points
# We remove the start point first
other_points = [p for p in points if p != start_point]

# Sort by angle. If angles are same, closest point comes first (though usually we want furthest for the hull, 
# but Graham scan typically handles collinear by keeping furthest. Let's keep it simple).
other_points.sort(key=lambda p: (polar_angle(start_point, p), distance_sq(start_point, p)))

sorted_points = [start_point] + other_points

### Step 2.3: Build the Hull
Iterate through the sorted points. For each point, check if adding it creates a "right turn" (concave). If so, remove the last added point until a "left turn" (convex) is formed.

In [ ]:
def cross_product(o, a, b):
    """Returns the cross product of vectors OA and OB.
    Positive means left turn, negative means right turn, zero means collinear.
    """
    return (a.x - o.x) * (b.y - o.y) - (a.y - o.y) * (b.x - o.x)

def graham_scan(sorted_pts):
    if len(sorted_pts) < 3:
        return sorted_pts

    hull = []
    
    for p in sorted_pts:
        while len(hull) >= 2 and cross_product(hull[-2], hull[-1], p) <= 0:
            hull.pop()
        hull.append(p)
        
    return hull

hull_points = graham_scan(sorted_points)
print(f"Convex Hull has {len(hull_points)} points.")

## 3. Visualization

Visualize the original points and the resulting convex hull polygon.

In [ ]:
viewer = Viewer()

# Add all original points as grey
for p in points:
    viewer.scene.add(p, pointcolor=Color.grey(), pointsize=10)

# Add start point as red
viewer.scene.add(start_point, pointcolor=Color.red(), pointsize=15)

# Create a closed polyline for the hull
if len(hull_points) > 0:
    # Close the loop
    hull_loop = hull_points + [hull_points[0]]
    polyline = Polyline(hull_loop)
    viewer.scene.add(polyline, linecolor=Color.blue(), linewidth=3)
    
    # Highlight hull vertices
    for p in hull_points:
        viewer.scene.add(p, pointcolor=Color.blue(), pointsize=12)

viewer.show()